In [1]:
import tempfile, zipfile
import os
import pandas as pd
import numpy as np

import TableParser
import TestTableParser

In [2]:
#########################
# repare the xlsx files #
#########################

def fix_xlsx(in_file):
    tmpfd, tmp = tempfile.mkstemp(dir=os.path.dirname(in_file))
    os.close(tmpfd)
    filename = '[Content_Types].xml'
    data = ''
    with zipfile.ZipFile(in_file, 'r') as zin:
        with zipfile.ZipFile(tmp, 'w') as zout:
            for item in zin.infolist():
                if item.filename != filename:
                    zout.writestr(item, zin.read(item.filename))
                else:
                    data = zin.read(filename).decode()
    os.remove(in_file)
    os.rename(tmp, in_file)
    data = data.replace('/xl/sharedStrings.xml', '/xl/SharedStrings.xml')
    with zipfile.ZipFile(in_file, mode='a', compression=zipfile.ZIP_DEFLATED) as zf:
        zf.writestr(filename, data)

In [3]:
fix_xlsx('file2.xlsx')

In [4]:
df = pd.read_excel('file2.xlsx')
df.head(11)

,,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 921,Unnamed: 922,Unnamed: 923,Unnamed: 924,Unnamed: 925,Unnamed: 926,Unnamed: 927,Unnamed: 928,Unnamed: 929,Unnamed: 930
0,"ПАО ""Птицефабрика ""Боровская""",NaN,NaN,Показатели продуктивного периода 1 Марта 201...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Цех №2,NaN,Птичник: Пт № 17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Показатели,NaN,ЕдИзм,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Дата на начало периода,NaN,NaN,01.03.2016,02.03.2016,03.03.2016,04.03.2016,05.03.2016,06.03.2016,07.03.2016,...,10.12.2018,11.12.2018,12.12.2018,13.12.2018,14.12.2018,15.12.2018,16.12.2018,17.12.2018,18.12.2018,19.12.2018
5,Наличие кур-несушек\nна начало периода,NaN,гол.,25936,25932,25932,25924,25917,25917,25908,...,30418,30417,30412,30406,30406,30396,30396,18229,6266,
6,Пало,норм,гол.,104,104,104,104,104,104,104,...,152,152,152,152,152,152,456,273,94,
7,NaN,факт,гол.,5,4,,8,7,,9,...,12,1,5,6,,10,,7,23,36
8,NaN,норм,%,0.40,0.40,0.40,0.40,0.40,0.40,0.40,...,0.50,0.50,0.50,0.50,0.50,0.50,1.50,1.50,1.50,1.50
9,NaN,факт,%,0.02,0.02,,0.03,0.03,,0.03,...,0.04,,0.02,0.02,,0.03,,0.04,0.37,


In [5]:
###############################################
# to find the x,y coordinate of row with date #
###############################################

date_row_num,date_col_num = 0,0
for i in range(df.shape[0]):
    if df.iloc[i,0] == 'Дата на начало периода': 
        date_row_num = i
#         print(date_row_num)
        break;
for j in range(1,df.shape[1]):
    if type(df.iloc[date_row_num,j])== type('string'):
        date_col_num = j
#         print(date_col_num)
        break;


In [6]:
date = df.iloc[date_row_num,date_col_num:].reset_index(drop=True) # <--- Series with date (01.03.2016;...)
table = df.iloc[date_row_num:,date_col_num:].reset_index(drop=True) # <--- all useful data

testTableParser =  TestTableParser.TestTableParser() # <--- import parser of date
tableParser = TableParser.TableParser(testTableParser) # <--- import parser of date

date = tableParser.parse(date)
table.iloc[0] = date[0]
table.iloc[0].reset_index(drop=True) # check of new date in data

0     2016-01-03
1     2016-02-03
2     2016-03-03
3     2016-04-03
4     2016-05-03
         ...    
923   2018-12-15
924   2018-12-16
925   2018-12-17
926   2018-12-18
927   2018-12-19
Name: 0, Length: 928, dtype: datetime64[ns]

In [7]:
table.columns = table.iloc[0]
table = table.iloc[1: , :]
table

,2016-01-03,2016-02-03,2016-03-03,2016-04-03,2016-05-03,2016-06-03,2016-07-03,2016-08-03,2016-09-03,2016-10-03,...,2018-10-12,2018-11-12,2018-12-12,2018-12-13,2018-12-14,2018-12-15,2018-12-16,2018-12-17,2018-12-18,2018-12-19
1,25936,25932,25932,25924,25917,25917,25908,25908,25899,25895,...,30418,30417,30412,30406,30406,30396,30396,18229,6266,
2,104,104,104,104,104,104,104,104,129,129,...,152,152,152,152,152,152,456,273,94,
3,5,4,,8,7,,9,,9,4,...,12,1,5,6,,10,,7,23,36
4,0.40,0.40,0.40,0.40,0.40,0.40,0.40,0.40,0.50,0.50,...,0.50,0.50,0.50,0.50,0.50,0.50,1.50,1.50,1.50,1.50
5,0.02,0.02,,0.03,0.03,,0.03,,0.03,0.02,...,0.04,,0.02,0.02,,0.03,,0.04,0.37,
6,3020.00,3000.00,3040.00,3000.00,3020.00,2000.00,2000.00,3080.00,3040.00,3100.00,...,3560.00,3600.00,3600.00,5140.00,,,,,,
7,25936,25932,25932,25924,25917,25917,25908,25908,25899,25895,...,30418,30417,30412,30406,30406,30396,30396,18229,6266,
8,117.0,117.0,117.0,117.0,117.0,117.0,117.0,117.0,117.0,117.0,...,117.0,117.0,117.0,117.0,117.0,117.0,117.0,117.0,117.0,117.0
9,11644.0,11568.7,11723.0,11572.3,11652.6,7716.9,7719.6,11888.2,11737.9,11971.4,...,11703.6,11835.5,11837.4,16904.6,,,,,,
10,,,,,,,,,,,...,,,,,,,,,,
